<a href="https://colab.research.google.com/github/minzzan/Colab/blob/main/HackathonTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from pathlib import Path

from torch import nn, optim
from torchvision import transforms, datasets

BATCH_SIZE = 64

---

**공개된 학습 데이셋 구조**

OpenDataset

　┖ Sequence01

　　┖ 00000.jpg ~ 00900.jpg

　┖ Sequence02

　　┖ 00000.jpg ~ 00900.jpg

　┖ Sequence03

　　┖ 00000.jpg ~ 00900.jpg

---

---

**공개된 학습용 데이터셋에 대한 설명을 위한 코드**

변수: ***dataset_path*** - 공개된 데이터셋 경로 

시퀀스 이름, 시퀀스 내 이미지 개수를 출력한다.

In [ ]:
dataset_path = '/content/drive/MyDrive/OpenDataset'
for seq_name in os.listdir(dataset_path):
  seq_path = os.path.join(dataset_path, seq_name)
  print('시퀀스 폴더:', seq_name, ', 시퀀스 이미지 개수:', len(os.listdir(seq_path)))

# for img_name in os.listdir(seq_path):
#      img_path = os.path.join(seq_path, img_name)
#      img = cv2.imread(img_path)
#      cv2_imshow(img)

---

---
**테스트 데이터셋 구조**

TestSequence

　┖ 00000.jpg ~ 00900.jpg

---

---

**테스트를 위한 경로 입력**

변수: ***test_seq_path*** - 테스트에 사용될 시퀀스의 경로

(제출된 코드에서 다음 경로만 변경하여 채점할 예정)

In [ ]:
test_seq_path = '/content/drive/MyDrive/OpenDataset/Sequence01'

---

---

**테스트 시퀀스의 이미지를 불러오는 class 정의**

테스트 시퀀스 안의 이미지들을 Tensor로 변환하여 내보내주는 Dataset class를 정의

**(각자 방법에 따라 class 내부를 수정해서 사용 )**

변수: ***imgPathes*** - 테스트 시퀀스 내 이미지들의 경로를 저장하는 list

함수: 

*__init__* - 시퀀스 내 이미지들의 경로를 imgPathes에 저장

*__getitem__* - 저장된 이미지들의 경로를 통해 이미지를 불러오고 텐서로 변환하여 출력한다. (PyTorch의 DataLoader, for 반복문 등에 의해 호출되는 함수)



In [ ]:
class testSequenceDataset(Dataset):
  def __init__(self, _test_seq_path):
    self.imgPathes = []

    # 시퀀스 내 이미지 경로들을 list에 저장(이름순 정렬)
    for _img_name in sorted(os.listdir(_test_seq_path)):
      img_path = os.path.join(_test_seq_path, _img_name)
      self.imgPathes.append(img_path)
  
  def __len__(self):
    return len(self.imgPathes)

  def __getitem__(self, idx):
    _img = cv2.imread(self.imgPathes[idx])
    _img = _img.astype(np.float32) / 255.0
    _img_tensor = torch.from_numpy(img.transpose(2, 0, 1))

    return _img_tensor


테스트 데이터셋 생성

In [ ]:
testDataset = testSequenceDataset(test_seq_path)
print('테스트 데이터셋의 이미지 개수:', len(testDataset.imgPathes))
print(testDataset.imgPathes)

테스트 데이터셋의 이미지 개수: 901
['/content/drive/MyDrive/OpenDataset/Sequence01/00000.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00001.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00002.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00003.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00004.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00005.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00006.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00007.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00008.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00009.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00010.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00011.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00012.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00013.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00014.jpg', '/content/drive/MyDrive/OpenDataset/Sequence01/00015.jpg', '/content/drive/MyDrive/OpenDatas

---

---

**문제풀이를 위한 카운트 변수 선언**

문제를 푸는 코드에서 해당 변수를 적절한 시점에 카운트한다.

In [ ]:
# 문제 변수
num_sec1 = 0       # section1을 통과한 차량 대수
num_sec2 = 0       # section2을 통과한 차량 대수
num_sec3 = 0       # section3을 통과한 차량 대수
num_sec4 = 0       # section4을 통과한 차량 대수
num_straight = 0   # 직진 차량 대수
num_left_turn = 0  # 좌회전 차량 대수
num_right_turn = 0 # 우회전 차량 대수

---

---

**
***문제풀이 코드 입력***
**

---

In [ ]:
class UltraSimpleParkingSpaceClassificationNet(nn.Module):
  def __init__(self):
    super(UltraSimpleParkingSpaceClassificationNet,self).__init__()
    num_sec1 = num_sec1+1
    self.layers = nn.Sequential(
      nn.Conv2d(3,32,3,padding=1),
      nn.ReLU()
    )
    self.fc_layers = nn.Sequential(
       nn.Linear(150*150*32, 100),
       nn.ReLU(),
       nn.Linear(100,2)
  ) 
  
  def forward(self, x):
    out = self.layers(x)
    out = out.view(BATCH_SIZE,-1)
    out = self.fc_layers(out)
    return out

In [ ]:
class ImageMultiFolder(Dataset):
  def __init__(self,data_root):
    self.imgNames = []

    for parkingLot in os.listdir(data_root):
      parkingLotPath = os.path.join(data_root, parkingLot)
      for statePath in os.listdir(parkingLotPath):
        img_filepath = os.path.join(parkingLotPath, statePath)
        for img in os.listdir(img_filepath):
          imgPath = os.path.join(img_filepath, img)
          self.imgNames.append(imgPath)

  def __len__(self):
    return len(self.imgNames)

  def __getitem__(self,idx):
    img = cv2.imread(self.imgNames[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    if 'busy' in self.imgNames[idx]:
      state = 0
    else:
      state = 1

    img_tensor = torch.from_numpy(img.transpose(2,0,1))
    return img_tensor, state

In [ ]:
#DIR_path = Path(test_seq_path)
#DIR_path = Path('/content/drive/MyDrive/OpenDataset/Sequence02')
DIR_path = Path('/content/drive/MyDrive/OpenDataset/Sequence03')
files_list = DIR_path.iterdir()

for f in files_list:
  img = cv2.imread(str(f))

  if img is None:
    print('unable to open',f)
  else:
    h,w,c = img.shape
    if h!=150 or w !=150 or c!=3:
      print('image',f,'has wrong dimension :', img.shape)
      resized = cv2.resize(img, dsize=(150,150), interpolation = cv2.INTER_AREA)
      cv2.imwrite(str(f),resized)

In [ ]:
image_datasets = testSequenceDataset(test_seq_path)

#왜 안되는거징!!
# for img, label in dataloader:
#      imgToShow = img.numpy()
#      print(imgToShow.shape)
#      print(label)
#      imgToShow = np.transpose(imgToShow[0],(1,2,0))
#      plt.imshow(imgToShow)
#      plt.show()

In [ ]:
train_size = int(len(image_datasets)*0.8)
test_size = len(image_datasets)-train_size
train_dataset, test_dataset = torch.utils.data.random_split(image_datasets,
                                                            [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers = 8,
                                           drop_last = True)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           num_workers = 8,
                                           drop_last = True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = UltraSimpleParkingSpaceClassificationNet().to(device)
model = model.train()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0002)

In [ ]:
num_epoch = 10

for i in range(num_epoch):
   for j, image in enumerate(train_loader):
     x=image.to(device)
     #y_ = label.to(device)

     optimizer.zero_grad()
     output = model.forward(x)
     loss = loss_func(output)
     loss.backward()
     optimizer.step()

     if j%100 == 0:
      print('epoch:', i, ', batch:',j,', loss is:', loss.data.item())

RuntimeError: ignored

In [ ]:
correct = 0
total = 0
model = model.eval()

with torch.no_grad():
  for image, label in test_loader:
    x = image.to(device)
    y_ = label.to(device)

    output = model.forward(x)
    _, output_index = torch.max(output,1)

    total += label.size(0)
    correct += (output_index == y_).sum().float()

  print('Total: {}, Correct: {}'.format(total,correct))
  print('Accuracy of Test Data: {}'.format(100*correct/total))
  

---

**문제 정답 출력 부분**

In [ ]:
print('문제 정답')
print(f'Section 1: {num_sec1}대')
print(f'Section 2: {num_sec2}대')
print(f'Section 3: {num_sec3}대')
print(f'Section 4: {num_sec4}대')
print(f'직진: {num_straight}대')
print(f'좌회전: {num_left_turn}대')
print(f'우회전: {num_right_turn}대')

문제 정답
Section 1: 0대
Section 2: 0대
Section 3: 0대
Section 4: 0대
직진: 0대
좌회전: 0대
우회전: 0대


---